<a href="https://colab.research.google.com/github/numthipka/CarDamageMaskRCNN/blob/master/mask_rcnn_damage_detection/TrainingModel_NoRsl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Trainning

## Prepare

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!git clone https://github.com/numthipka/CarDamageMaskRCNN.git

Cloning into 'CarDamageMaskRCNN'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 714 (delta 31), reused 19 (delta 9), pack-reused 658
Receiving objects: 100% (714/714), 220.70 MiB | 14.22 MiB/s, done.
Resolving deltas: 100% (76/76), done.
Checking out files: 100% (430/430), done.


In [3]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [4]:
!pip install -r '/content/CarDamageMaskRCNN/mask_rcnn_damage_detection/requirements.txt'

     |████████████████████████████████| 377.0MB 40kB/s 
     |████████████████████████████████| 337kB 63.9MB/s 
     |████████████████████████████████| 3.2MB 59.1MB/s 
     |████████████████████████████████| 491kB 58.9MB/s 
     |████████████████████████████████| 204kB 65.6MB/s 
     |████████████████████████████████| 163kB 62.9MB/s 
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [5]:
import os
os.chdir('/content/CarDamageMaskRCNN/mask_rcnn_damage_detection')

## 1 Import Dependencies 

In [6]:
# import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
#numthip add on 20200721 : in order to dont show the warning 
import warnings
warnings.filterwarnings("ignore")

import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

# numthip add on 20200719 for img augmentation train
import imgaug as ia
from imgaug import augmenters as iaa

# Root directory of the project
# ROOT_DIR = os.path.abspath("../../")
# ROOT_DIR = os.path.abspath("../")
# 20200627
ROOT_DIR = os.getcwd()

## 2 Constant

In [7]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Cocmmand
DEFAULT_COMMAND = "train"
print("Command : ",DEFAULT_COMMAND)

# Weight
# 20200716 numthip k. comment ::>> DEFAULT_WEIGHT = "last"
DEFAULT_WEIGHT = "coco"
print("WEIGHT : ",DEFAULT_WEIGHT)

# Path to trained weights file
# COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
COCO_WEIGHTS_PATH = '/content/gdrive/My Drive/mask_rcnn/mask_rcnn_coco.h5'
print("Coco Weights Path : ",COCO_WEIGHTS_PATH)

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
print("Log path : ", DEFAULT_LOGS_DIR)

# Directory of data
# DEFAULT_DATASET = os.path.join(os.getcwd(), "customImages")
DEFAULT_DATASET = '/content/gdrive/My Drive/mask_rcnn/customImages'
print("Dataset path : ", DEFAULT_DATASET)

# Path to trained weights file
# LAST_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
LAST_WEIGHTS_PATH = '/content/gdrive/My Drive/mask_rcnn/mask_rcnn_coco.h5'

# os.path.join(os.path.abspath("../logs/damage20200628T1311"), "mask_rcnn_damage_0010.h5")
#print("Last Weights Path : ",LAST_WEIGHTS_PATH)
 
    
# Numthip Add constnt epoch 
# number of epoch for training 
EPOCHS_CONST = 40


Command :  train
WEIGHT :  coco
Coco Weights Path :  /content/gdrive/My Drive/mask_rcnn/mask_rcnn_coco.h5
Log path :  /content/CarDamageMaskRCNN/mask_rcnn_damage_detection/logs
Dataset path :  /content/gdrive/My Drive/mask_rcnn/customImages


Using TensorFlow backend.


## 3 Configurations

In [8]:
class CustomConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "damage"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
#     IMAGES_PER_GPU = 2 Numthip
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + toy

    # Number of training steps per epoch 
    # 20200716 : numthip changes from 100  to 200 
    STEPS_PER_EPOCH = 600

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9


## 4 Dataset

In [ ]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("damage", 1, "damage")

        # Train or validation dataset?
        assert subset in ["train", "val", "test"]
        # dataset_dir = os.path.join(dataset_dir, subset)
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions'].values()]

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "damage",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "damage":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "damage":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

## 5 Image Augmentation 
only horizontol flip here

In [ ]:
#2020719 - Numthip added  augmentationof the images set to be horizontal flip 
#augmentation = iaa.Sequential([
#    iaa.Fliplr(0.5) # only horizontal flip here
#])
augmentation = iaa.Sequential([
    iaa.OneOf([ ## rotate
        iaa.Affine(rotate=0),
        iaa.Affine(rotate=90),
    ]),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.OneOf([ ## brightness or contrast
        iaa.Multiply((0.9, 1.1)),
        iaa.ContrastNormalization((0.9, 1.1)),
    ]),
    iaa.OneOf([ ## blur or sharpen
        iaa.GaussianBlur(sigma=(0.0, 0.1)),
        iaa.Sharpen(alpha=(0.0, 0.1)),
    ]),
])


In [ ]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(DEFAULT_DATASET, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(DEFAULT_DATASET, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
      # 20200716 : numthip changes from 100  to 200 
        #model.train(dataset_train, dataset_val,
        #       learning_rate=config.LEARNING_RATE,
        #      epochs=10,
        #     layers='heads')
        #,#augmentation=augmentation)
        
    ##Numthip Add : 20200726 
    ##add call back for mAP Cal as Link: "https://github.com/matterport/Mask_RCNN/issues/1839" 
    ##custom_callbacks
    
    # model_inference = modellib.MaskRCNN(mode="inference", config=config, model_dir=DEFAULT_LOGS_DIR)
    
    # mean_average_precision_callback = modellib.MeanAveragePrecisionCallback(model
    #                                                                         , model_inference
    #                                                                         , dataset_val
    #                                                                         , calculate_map_at_every_X_epoch=5
    #                                                                         , verbose=1)
        
    model.train(dataset_train, dataset_val,
             learning_rate=config.LEARNING_RATE,
             epochs=EPOCHS_CONST,
             layers='all')
             #,augmentation=augmentation)
             #,
             #custom_callbacks=[mean_average_precision_callback])
  

In [ ]:
def color_splash(image, mask):
    """Apply color splash effect.
    image: RGB image [height, width, 3]
    mask: instance segmentation mask [height, width, instance count]

    Returns result image.
    """
    # Make a grayscale copy of the image. The grayscale copy still
    # has 3 RGB channels, though.
    gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image)) * 255
    # We're treating all instances as one, so collapse the mask into one layer
    mask = (np.sum(mask, -1, keepdims=True) >= 1)
    # Copy color pixels from the original color image where mask is set
    if mask.shape[0] > 0:
        splash = np.where(mask, image, gray).astype(np.uint8)
    else:
        splash = gray
    return splash

## 6 Training

In [ ]:
if __name__ == '__main__':

    # Configurations
    if DEFAULT_COMMAND == "train":
        config = CustomConfig()
    else:
        class InferenceConfig(CustomConfig):
            # Set batch size to 1 since we'll be running inference on
            # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
        config = InferenceConfig()
    config.display()

    # Create model
    if DEFAULT_COMMAND == "train":
        model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)
    else:
        model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

    # Select weights file to load
    if DEFAULT_WEIGHT.lower() == "coco":
        weights_path = COCO_WEIGHTS_PATH
        # Download weights file
        if not os.path.exists(weights_path):
            utils.download_trained_weights(weights_path)
    elif DEFAULT_WEIGHT.lower() == "last":
        # Find last trained weights
        weights_path = LAST_WEIGHTS_PATH
    elif DEFAULT_WEIGHT.lower() == "imagenet":
        # Start from ImageNet trained weights
        weights_path = model.get_imagenet_weights()
    else:
        weights_path = DEFAULT_WEIGHT

    # Load weights
    print("Loading weights ", weights_path)
    if DEFAULT_WEIGHT.lower() == "coco":
        # Exclude the last layers because they require a matching
        # number of classes
        model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
    else:
        model.load_weights(weights_path, by_name=True)

    # Train
    train(model)
    
    history = model.keras_model.history.history

## 7 Evaluate Model Loss and Val Loss

In [ ]:
# numthip add chart train loss and valid loss 

epochs = range(EPOCHS_CONST)

plt.figure(figsize=(18, 6))

plt.subplot(131)
plt.title('loss')
plt.plot(epochs, history['loss'], label="train loss")
plt.plot(epochs, history['val_loss'], label="valid loss")
plt.legend()
plt.subplot(132)
plt.title('mrcnn_class_loss')
plt.plot(epochs, history['mrcnn_class_loss'], label="train class loss")
plt.plot(epochs, history['val_mrcnn_class_loss'], label="valid class loss")
plt.legend()
plt.subplot(133)
plt.title('mrcnn_mask_loss')
plt.plot(epochs, history['mrcnn_mask_loss'], label="train mask loss")
plt.plot(epochs, history['val_mrcnn_mask_loss'], label="valid mask loss")
plt.legend()

plt.show()

best_epoch = np.argmin(history["val_loss"]) + 1
print("Best epoch: ", best_epoch)
print("Valid loss: ", history["val_loss"][best_epoch-1])

best_epoch2 = np.argmin(history["val_loss"])
print("Best Epoch:", best_epoch2 + 1, history["val_loss"][best_epoch2])


In [ ]:
# numthip add to print  train loss and valid loss 
#epochs = range(1,len(next(iter(history.values())))+1)
#pd.DataFrame(history, index=epochs)

## Backup Logs

In [ ]:
cp -r /content/CarDamageMaskRCNN/mask_rcnn_damage_detection/logs/damage20200729T0816 '/content/gdrive/My Drive/mask_rcnn'

# Inspect Custom Trained Model
Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

## Configurations

In [ ]:
#from mrcnn.config import Config
#from mrcnn import model as modellib, utils
#from mrcnn.config2 import Config # Numthip change config file to config2.py that parameter will be Resnet50
config = custom.CustomConfig()
# custom_DIR = os.path.join(ROOT_DIR, "mask_rcnn_damage_detection\customImages")
CUSTOM_DIR = DEFAULT_DATASET

print ("CUSTOM_DIR",CUSTOM_DIR)

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Notebook Preferences

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Validation Dataset

In [ ]:
# Load validation dataset
dataset = custom.CustomDataset()
dataset.load_custom(CUSTOM_DIR, "test")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

## Load Model

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", CUSTOM_WEIGHTS_PATH)
model.load_weights(CUSTOM_WEIGHTS_PATH, by_name=True)

In [ ]:
from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)

## Evaluate Model mAP

In [ ]:
from mrcnn.utils import compute_ap
class EvalImage():
  def __init__(self,dataset,model,cfg):
    self.dataset = dataset
    self.model   = model
    self.cfg     = cfg

 
  def evaluate_model(self , len = 50):
    APs = list()
    precisions_dict = {}
    recall_dict     = {}
    for index,image_id in enumerate(self.dataset.image_ids):
      if(index > len):
         break; 
      # load image, bounding boxes and masks for the image id
      image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(self.dataset, self.cfg,image_id, use_mini_mask=False)
      # convert pixel values (e.g. center)
      #scaled_image = modellib.mold_image(image, self.cfg)
      # convert image into one sample
      sample = np.expand_dims(image, 0)
     # print(len(image))
      # make prediction
      yhat = self.model.detect(sample, verbose=1)
      # extract results for first sample
      r = yhat[0]
      # calculate statistics, including AP
      AP, precisions, recalls, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
      precisions_dict[image_id] = np.mean(precisions)
      recall_dict[image_id] = np.mean(recalls)
      # store
      APs.append(AP)

    # calculate the mean AP across all images
    mAP = np.mean(APs)
    return mAP,precisions_dict,recall_dict

In [ ]:
# dataset_val = Dataset() 
# dataset_val.load_data_set("onion","val_data")
# dataset_val.prepare()

dataset_test = CustomDataset()
dataset_test.load_custom(DEFAULT_DATASET, "test")
dataset_test.prepare()

In [ ]:
# config = DataConfig()
eval = EvalImage(dataset_test,model,config)

In [ ]:
eval.evaluate_model()

In [ ]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
import warnings

warnings.filterwarnings("ignore")
# Root directory of the project
ROOT_DIR = os.getcwd()
# ROOT_DIR = os.path.abspath("../")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import custom 

%matplotlib inline 

# Directory to save logs and trained model

MODEL_DIR = os.path.join(ROOT_DIR, "logs/damage20200729T0816")  

# custom_WEIGHTS_PATH = os.path.join(MODEL_DIR, "mask_rcnn_damage_0010.h5")
CUSTOM_WEIGHTS_PATH = os.path.join(MODEL_DIR, "mask_rcnn_damage_0008.h5")


print("root path:", ROOT_DIR)
print("MODEL_DIR:", MODEL_DIR)
print("CUSTOM_WEIGHTS_PATH:", CUSTOM_WEIGHTS_PATH)

## Run Detection on Images

In [ ]:
#export TF_FORCE_GPU_ALLOW_GROWTH=True
#config.gpu_options.allow_growth = True

In [ ]:
image_id = random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))

# Run object detection
results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

## Plot precision-recall curve

In [ ]:
# Author Numthip k. on 20200706
# For: Draw precision-recall curve
AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize.plot_precision_recall(AP, precisions, recalls)
# Add title and axis names
#Author : numthip 
#for: add chart title 
plt.xlabel('recall')
plt.ylabel('precision')

In [ ]:
# Grid of ground truth objects and their predictions
visualize.plot_overlaps(gt_class_id, r['class_ids'], r['scores'],
                        overlaps, dataset.class_names)

## Compute mAP @ IoU=50 on Batch of Images

In [ ]:
def compute_batch_ap(image_ids):
    APs = []
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        # Compute AP
        r = results[0]
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                              r['rois'], r['class_ids'], r['scores'], r['masks'])
        APs.append(AP)
    return APs

# Pick a set of random images
# 20200719 Numthip revise len(dataset.image_ids) from 10
image_ids = np.random.choice(dataset.image_ids, len(dataset.image_ids))
print (image_ids)
APs = compute_batch_ap(image_ids)

print("mAP @ IoU=50: ", np.mean(APs))

## Test Model with The real test data

In [ ]:

# Load validation dataset
# real_test_dir = custom.CustomDataset()
# real_test_dir.load_custom(CUSTOM_DIR, "test")

# Must call before using the dataset
#real_test_dir.prepare()
# real_test_dir = custom.CustomConfig()
real_test_dir = os.path.join(CUSTOM_DIR, "test")
#real_test_dir = '..\customImages\val'

print (real_test_dir) 

#image_path = "dataset/{0:04}.JPEG".format(random_file) # Input Image path
image_paths = []
for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))

for image_path in image_paths:
    img = skimage.io.imread(image_path)
    img_arr = np.array(img)
    results = model.detect([img_arr], verbose=1)
    r = results[0]
    visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], figsize=(7,7))